# Initial Analysis to see what's interesting, has no relevance in the analysis of paper.

In [1]:
# Constants used in the program:
# URL_DATA_FOLDER is where all the data  is saved.
URL_DATA_FOLDER = "./"

# ALEXA TOP 1M file
ALEXA_TOP_1M_FILEPATH = "./alexa-top1m-2020-09-03_0900_UTC.csv"

# CDF_FILE_OUTPUT is where the CDF graph will be outputted to
CDF_FILE_OUTPUT = 'test.pdf'

In [2]:
import json 
import os

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [4]:
# Alexa top 1m sorted by popularity
popularity_list = []
with open(ALEXA_TOP_1M_FILEPATH, 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        popularity_list.append(line.split(",")[1])

In [5]:
print(popularity_list[:100])

['google.com', 'youtube.com', 'tmall.com', 'baidu.com', 'qq.com', 'sohu.com', 'facebook.com', 'taobao.com', 'yahoo.com', 'jd.com', '360.cn', 'amazon.com', 'zoom.us', 'wikipedia.org', 'weibo.com', 'sina.com.cn', 'live.com', 'reddit.com', 'microsoft.com', 'xinhuanet.com', 'netflix.com', 'office.com', 'okezone.com', 'vk.com', 'microsoftonline.com', 'csdn.net', 'instagram.com', 'myshopify.com', 'alipay.com', 'yahoo.co.jp', 'github.com', 'naver.com', 'zhanqi.tv', 'panda.tv', 'bongacams.com', 'twitch.tv', 'stackoverflow.com', 'bing.com', 'google.com.hk', 'ebay.com', 'canva.com', 'aliexpress.com', 'amazon.in', 'tianya.cn', 'tribunnews.com', 'apple.com', 'amazon.co.jp', 'adobe.com', 'google.co.in', 'china.com.cn', 'livejasmin.com', 'chaturbate.com', 'yandex.ru', 'aparat.com', 'twitter.com', 'dropbox.com', 'mail.ru', 'linkedin.com', 'yy.com', 'instructure.com', 'msn.com', 'pornhub.com', 'huanqiu.com', 'google.cn', 'wordpress.com', 'sogou.com', '1688.com', 'whatsapp.com', 'google.co.jp', '17ok.c

In [6]:
# From output:
# website_url + "$$" + json.dumps(tracker_urls) + "$$" + json.dumps(ads_urls) + "$$" + json.dumps(other_urls)
websites = {}
for root, dirs, files in os.walk(URL_DATA_FOLDER, topdown=False):
    for name in files:
        # goes through the data generated from the branch-analysis script
        if('urls' in name):
            filePath = os.path.join(root, name)
            print(filePath)
            with open(filePath, 'r') as f:
                lines = f.read().splitlines()
                for line in lines:
                    fields = line.split("$${")
                    url = fields[0]
#                     print(url)
                    trackers = json.loads("{" + fields[1])
                    ads = json.loads("{" + fields[2])
                    others = json.loads("{" + fields[3])
                    websites[url] = {}
                    websites[url]["trackers"] = trackers
                    websites[url]["ads"] = ads
                    websites[url]["others"] = others
                    

./13\output\urls_0.txt
./13\output\urls_1.txt
./13\output\urls_2.txt
./13\output\urls_3.txt
./13\output\urls_4.txt
./13\output\urls_5.txt
./13\output\urls_6.txt
./13\output\urls_7.txt
./14\output\urls_0.txt
./14\output\urls_1.txt
./14\output\urls_2.txt
./14\output\urls_3.txt
./14\output\urls_4.txt
./14\output\urls_5.txt
./14\output\urls_6.txt
./14\output\urls_7.txt
./15\output\urls_0.txt
./15\output\urls_1.txt
./15\output\urls_2.txt
./15\output\urls_3.txt
./15\output\urls_4.txt
./15\output\urls_5.txt
./15\output\urls_6.txt
./15\output\urls_7.txt


In [7]:
# Check if ad and tracker counts were > 1. Since it's none, this demonstrates that each tracker 
# and ad resource with specific parameters are downloaded once. There could be multiple instances of a single tracker
# but different parameters.
for website in websites:
    for tracker in websites[website]["trackers"]:
        if websites[website]["trackers"][tracker]["count"] > 1:
            print(website, "tracker count > 1")
    for ad in websites[website]["ads"]:
        if websites[website]["ads"][ad]["count"] > 1:
            print(website, "ad count > 1")

In [53]:
# Popular 100 Websites and their trackers and ads. This constructs the dictionary of the popular 100, then finds their 
# aggregates the data
popular_100_trackers = {}
popular_100_ads = {}
print("Top 100 that were not included:")
for popular_100_website in popularity_list[:100]:
    if popular_100_website not in websites:
        print(popular_100_website)
    else:
        for tracker in websites[popular_100_website]["trackers"]:
            old_tracker = tracker
            tracker = tracker.split("?")[0]
            if tracker not in popular_100_trackers:
                popular_100_trackers[tracker] = {}
                popular_100_trackers[tracker]["count"] = 1
                popular_100_trackers[tracker]["bytes"] = websites[popular_100_website]["trackers"][old_tracker]["total_bytes"]
            else:
                popular_100_trackers[tracker]["count"] += 1
                popular_100_trackers[tracker]["bytes"] += websites[popular_100_website]["trackers"][old_tracker]["total_bytes"]
        for ad in websites[popular_100_website]["ads"]:
            old_ad = ad
            ad = ad.split("?")[0]
            if ad not in popular_100_ads:
                popular_100_ads[ad] = {}
                popular_100_ads[ad]["count"] = 1
                popular_100_ads[ad]["bytes"] = websites[popular_100_website]["ads"][old_ad]["total_bytes"]
            else:
                popular_100_ads[ad]["count"] += 1
                popular_100_ads[ad]["bytes"] += websites[popular_100_website]["ads"][old_ad]["total_bytes"]


Top 100 that were not included:
amazon.com
microsoftonline.com
amazon.in
amazon.co.jp
amazon.co.uk


In [64]:
popular_100_tracker_domains = []
for tracker in popular_100_trackers:
    if(popular_100_trackers[tracker]["count"] >= 5):
        print(popular_100_trackers[tracker]["count"], tracker)
        popular_100_tracker_domains.append(tracker)

34 https://btrace.qq.com/kvcollect
5 https://dp3.qq.com/stdlog
5 https://track.sohu.com/se.gif
63 https://i.go.sohu.com/count/v
8 https://pos.baidu.com/scim
123 https://track.sohu.com/sa.gif
13 https://www.googletagmanager.com/gtag/js
5 https://geo.yahoo.com/p
25 https://www.google-analytics.com/analytics.js
18 https://www.googletagmanager.com/gtm.js
6 https://www.google-analytics.com/gtm/js
26 https://stats.g.doubleclick.net/j/collect
15 https://saxn.sina.com.cn/view
6 https://c1.microsoft.com/c.gif
12 https://dpm.demdex.net/id
7 https://c.bing.com/c.gif
8 https://sb.scorecardresearch.com/beacon.js
12 https://certify.alexametrics.com/atrk.gif
7 https://hm.baidu.com/hm.gif
10 https://logql.yahoo.co.jp/v1/public/yql
6 https://ib.adnxs.com/bounce
6 https://yads.yjtag.yahoo.co.jp/tag
5 https://ib.adnxs.com/getuid
5 https://nv.veta.naver.com/fxshow
9 https://d31qbv1cthcecs.cloudfront.net/atrk.js
16 https://www.bing.com/fd/ls/l
10 https://bat.bing.com/bat.js
8 https://www.googleadservices.c

In [60]:
popular_100_ad_domains = []
for ad in popular_100_ads:
    if(popular_100_ads[ad]["count"] >= 5):
        print(ad)
        print(popular_100_ads[ad]["count"])
        popular_100_ad_domains.append(ad)

https://securepubads.g.doubleclick.net/gpt/pubads_impl_2020091601.js
7
https://c.amazon-adsystem.com/aax2/apstag.js
5
https://securepubads.g.doubleclick.net/gampad/ads
23
https://tpc.googlesyndication.com/safeframe/1-0-37/html/container.html
15
https://securepubads.g.doubleclick.net/tag/js/gpt.js
7
https://securepubads.g.doubleclick.net/pcs/view
51
https://pagead2.googlesyndication.com/getconfig/sodar
7
https://tpc.googlesyndication.com/sodar/sodar2.js
7
https://ad2.apx.appier.net/v1/prebid/bid
10
https://kompascybermedia-d.openx.net/w/1.0/arj
5
https://pagead2.googlesyndication.com/pcs/activeview
6
https://pagead2.googlesyndication.com/pagead/gen_204
5
https://titan.babytree.com/delivery/ajsc.php
5
https://titan.babytree.com/delivery/lg.php
5


In [61]:
# This code will print out each popular 100 website and their trackers and their counts
count = 0
for popular_100_website in popularity_list[:100]:
    for tracker_domain in popular_100_tracker_domains:
        if popular_100_website in websites:
            tracker_count = 0
            for tracker_domain_2 in websites[popular_100_website]["trackers"]:
                tracker_domain_2 = tracker_domain_2.split("?")[0]
                if tracker_domain == tracker_domain_2:
                    tracker_count += 1
#                     print(count, popular_100_website, tracker_domain_2, tracker_count)
    count += 1

In [62]:
# this code will grab the unqiue trackers, whose trackers count >= 10 and only show up on a single website.
unique_trackers = set()
count = 0

for popular_100_website in popularity_list[:100]:
    for tracker_domain in popular_100_tracker_domains:
        if popular_100_website in websites:
            tracker_count = 0
            for tracker_domain_2 in websites[popular_100_website]["trackers"]:
                tracker_domain_2 = tracker_domain_2.split("?")[0]
                if tracker_domain == tracker_domain_2:
                    tracker_count += 1
                    if tracker_count == popular_100_trackers[tracker_domain]["count"]:
                        unique_trackers.add(tracker_domain)
                        print(count, popular_100_website, tracker_domain_2, tracker_count)
                        total_tracker_for_web = 0
#                         for tracker_in_100 in websites[popular_100_website]["trackers"]:
#                             total_tracker_for_web +=  websites[popular_100_website]["trackers"][tracker_in_100]["count"]
#                         print(tracker_count / total_tracker_for_web)
    count += 1

4 qq.com https://btrace.qq.com/kvcollect 34
4 qq.com https://dp3.qq.com/stdlog 5
5 sohu.com https://track.sohu.com/se.gif 5
5 sohu.com https://i.go.sohu.com/count/v 63
5 sohu.com https://pos.baidu.com/scim 8
5 sohu.com https://track.sohu.com/sa.gif 123
8 yahoo.com https://geo.yahoo.com/p 5
15 sina.com.cn https://saxn.sina.com.cn/view 15
29 yahoo.co.jp https://logql.yahoo.co.jp/v1/public/yql 10
29 yahoo.co.jp https://yads.yjtag.yahoo.co.jp/tag 6
31 naver.com https://nv.veta.naver.com/fxshow 5
37 bing.com https://www.bing.com/fd/ls/l 16
41 aliexpress.com https://gj.mmstat.com/ae.pc_ctr.statweb_ae_ctr 5
58 yy.com https://mlog.hiido.com/j.gif 9
60 msn.com https://srtb.msn.com/notify/served 9
66 1688.com https://retcode.taobao.com/r.png 36
71 amazonaws.com https://fls-na.amazon.com/1/aws-mktg/1/OE/ 9
77 rakuten.co.jp https://rat.rakuten.co.jp/ 6
81 kompas.com https://csi.gstatic.com/csi 17
99 walmart.com https://beacon.walmart.com/rum.gif 16


In [63]:
count = 0
for popular_100_website in popularity_list[:100]:
    for ad_domain in popular_100_ad_domains:
        if popular_100_website in websites:
            for ad_domain_2 in websites[popular_100_website]["ads"]:
                ad_domain_2 = ad_domain_2.split("?")[0]
                if ad_domain == ad_domain_2:
                    print(count, popular_100_website, ad_domain)
    count += 1

14 weibo.com https://securepubads.g.doubleclick.net/gpt/pubads_impl_2020091601.js
17 reddit.com https://c.amazon-adsystem.com/aax2/apstag.js
22 okezone.com https://securepubads.g.doubleclick.net/gampad/ads
22 okezone.com https://tpc.googlesyndication.com/safeframe/1-0-37/html/container.html
29 yahoo.co.jp https://securepubads.g.doubleclick.net/gampad/ads
29 yahoo.co.jp https://tpc.googlesyndication.com/safeframe/1-0-37/html/container.html
29 yahoo.co.jp https://securepubads.g.doubleclick.net/tag/js/gpt.js
29 yahoo.co.jp https://securepubads.g.doubleclick.net/pcs/view
29 yahoo.co.jp https://pagead2.googlesyndication.com/getconfig/sodar
29 yahoo.co.jp https://tpc.googlesyndication.com/sodar/sodar2.js
35 twitch.tv https://securepubads.g.doubleclick.net/gpt/pubads_impl_2020091601.js
35 twitch.tv https://c.amazon-adsystem.com/aax2/apstag.js
39 ebay.com https://securepubads.g.doubleclick.net/gpt/pubads_impl_2020091601.js
39 ebay.com https://securepubads.g.doubleclick.net/gampad/ads
39 ebay.c